In [ ]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-1962, -365, 62)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if i == 0:
                allstock = stockdata
                # print(allstock)
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>50MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '50MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS EMA250rate>75'] = (allstock.loc[id, 'RS EMA250rate']>75)
            allstock.loc[id, 'RS EMA50rate>80'] = (allstock.loc[id, 'RS EMA50rate'] > 80)
            allstock.loc[id, 'RS EMA20rate>80'] = (allstock.loc[id, 'RS EMA20rate'] > 80)
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 100*1000)
            allstock.loc[id, 'new_template'] = all(allstock.loc[id, ['year low sort', 'RS EMA20rate>80', 'RS EMA250rate>75', 'Price>50MA', 'Volume 50MA>100k']])
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    apexstock = allstock.loc[allstock['new_template'], ['產業別', 'RS 250rate', 'RS 50rate', 'RS 20rate', 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA',
                                                        'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(len(allstock.loc[allstock['new_template']].index))
    print(str(day))
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/history_choosen/' + str(day).split(' ')[0] + 'RS選股(EMA)' + '.xlsx', encoding='utf-8-sig')


    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+365))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
        if i == 0:
            rocp_df = stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP']
            # rocp_df = rocp_df.loc[(rocp_df.index<day2)]
        else:
            rocp = pd.Series(stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP'], name = str(ID))
            rocp_df = pd.concat([rocp, rocp_df], axis=1)

    roc_df = rocp_df/100
    roc_df = roc_df+1   
    season1_profit = abs(roc_df.iloc[:42]).prod()
    season2_profit = abs(roc_df.iloc[42:84]).prod()
    season3_profit = abs(roc_df.iloc[84:126]).prod()
    season4_profit = abs(roc_df.iloc[126:168]).prod()
    season5_profit = abs(roc_df.iloc[168:210]).prod()
    season6_profit = abs(roc_df.iloc[210:]).prod()
    all_profit = abs(roc_df.iloc[:]).prod()
    season1_profit = season1_profit-1
    season2_profit = season2_profit-1
    season3_profit = season3_profit-1
    season4_profit = season4_profit-1
    season5_profit = season5_profit-1
    season6_profit = season6_profit-1
    all_profit = all_profit-1
    season1_profit = season1_profit.sum()
    season2_profit = season2_profit.sum()
    season3_profit = season3_profit.sum()
    season4_profit = season4_profit.sum()
    season5_profit = season5_profit.sum()
    season6_profit = season6_profit.sum()
    all_profit = all_profit.sum()
    profit = {'season1_profit':season1_profit,
            'season2_profit':season2_profit,
            'season3_profit':season3_profit,
            'season4_profit':season4_profit,
            'season5_profit':season3_profit,
            'season6_profit':season4_profit,
            'all_profit':all_profit,
            'number of stock':len(allstock.loc[allstock['new_template']].index)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

In [3]:
avg_profit_df3 = pd.DataFrame([], columns=all_df.columns)
for ind in all_df.index.values:
    avg_profit_df3.loc[ind] = 100*all_df.loc[ind]/all_df.loc[ind, 'number of stock']
avg_profit_df3.loc['total'] = avg_profit_df3.sum()
display(avg_profit_df3)

,season1_profit,season2_profit,season3_profit,season4_profit,season5_profit,season6_profit,all_profit,number of stock
2022-06-17 00:00:00,5.051134,-10.494660,13.184949,8.392633,13.184949,8.392633,30.712988,100.0
2022-04-18 00:00:00,-9.213736,3.344685,-9.275742,12.461088,-9.275742,12.461088,8.676901,100.0
2022-02-14 00:00:00,-0.558805,-3.341566,-0.132781,-12.209894,-0.132781,-12.209894,-3.350434,100.0
2021-12-13 00:00:00,-4.968352,-10.058267,-3.344245,7.042507,-3.344245,7.042507,-15.948079,100.0
2021-10-12 00:00:00,8.882910,-3.014714,-0.032959,-8.734773,-0.032959,-8.734773,-10.474087,100.0
2021-08-11 00:00:00,-1.264035,10.898360,-0.747253,-1.547361,-0.747253,-1.547361,-2.464498,100.0
2021-06-10 00:00:00,11.112004,-10.027091,17.249973,-2.406935,17.249973,-2.406935,8.962369,100.0
2021-04-09 00:00:00,-3.468964,11.689067,-9.469890,17.387131,-9.469890,17.387131,5.332209,100.0
2021-02-17 00:00:00,21.837272,0.285416,-2.652125,1.679292,-2.652125,1.679292,30.896354,100.0
2020-12-07 00:00:00,3.460317,38.192134,-4.344083,2.674041,-4.344083,2.674041,59.086226,100.0


In [ ]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-5000, -30, 62)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if i == 0:
                allstock = stockdata
                # print(allstock)
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>50MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '50MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS EMA250rate>75'] = (allstock.loc[id, 'RS EMA250rate']>75)
            allstock.loc[id, 'RS EMA50rate>80'] = (allstock.loc[id, 'RS EMA50rate'] > 80)
            allstock.loc[id, 'RS EMA20rate>80'] = (allstock.loc[id, 'RS EMA20rate'] > 80)
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 100*1000)
            allstock.loc[id, 'new_template'] = all(allstock.loc[id, ['year low sort', 'RS EMA20rate>80', 'RS EMA250rate>75', 'Price>50MA', 'Volume 50MA>100k']])
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    apexstock = allstock.loc[allstock['new_template'], ['產業別', 'RS 250rate', 'RS 50rate', 'RS 20rate', 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA',
                                                        'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(len(allstock.loc[allstock['new_template']].index))
    print(str(day))
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/history_choosen/' + str(day).split(' ')[0] + 'RS選股(EMA)' + '.xlsx', encoding='utf-8-sig')


    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+365))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
        if i == 0:
            rocp_df = stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP']
            # rocp_df = rocp_df.loc[(rocp_df.index<day2)]
        else:
            rocp = pd.Series(stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP'], name = str(ID))
            rocp_df = pd.concat([rocp, rocp_df], axis=1)

    roc_df = rocp_df/100
    roc_df = roc_df+1   
    season1_profit = abs(roc_df.iloc[:42]).prod()
    season2_profit = abs(roc_df.iloc[42:84]).prod()
    season3_profit = abs(roc_df.iloc[84:126]).prod()
    season4_profit = abs(roc_df.iloc[126:168]).prod()
    season5_profit = abs(roc_df.iloc[168:210]).prod()
    season6_profit = abs(roc_df.iloc[210:]).prod()
    all_profit = abs(roc_df.iloc[:]).prod()
    season1_profit = season1_profit-1
    season2_profit = season2_profit-1
    season3_profit = season3_profit-1
    season4_profit = season4_profit-1
    season5_profit = season5_profit-1
    season6_profit = season6_profit-1
    all_profit = all_profit-1
    season1_profit = season1_profit.sum()
    season2_profit = season2_profit.sum()
    season3_profit = season3_profit.sum()
    season4_profit = season4_profit.sum()
    season5_profit = season5_profit.sum()
    season6_profit = season6_profit.sum()
    all_profit = all_profit.sum()
    profit = {'season1_profit':season1_profit,
            'season2_profit':season2_profit,
            'season3_profit':season3_profit,
            'season4_profit':season4_profit,
            'season5_profit':season3_profit,
            'season6_profit':season4_profit,
            'all_profit':all_profit,
            'number of stock':len(allstock.loc[allstock['new_template']].index)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

In [6]:
avg_profit_df4 = pd.DataFrame([], columns=all_df.columns)
for ind in all_df.index.values:
    avg_profit_df4.loc[ind] = 100*all_df.loc[ind]/all_df.loc[ind, 'number of stock']
avg_profit_df4.loc['total'] = avg_profit_df4.sum()
display(avg_profit_df4)

,season1_profit,season2_profit,season3_profit,season4_profit,season5_profit,season6_profit,all_profit,number of stock
2023-06-26 00:00:00,-2.477569,0.000000,0.000000,0.000000,0.000000,0.000000,-2.477569,100.0
2023-04-24 00:00:00,9.078621,-1.493184,0.000000,0.000000,0.000000,0.000000,7.376312,100.0
2023-02-20 00:00:00,8.469134,13.287433,-3.237114,0.000000,-3.237114,0.000000,18.403126,100.0
2022-12-20 00:00:00,14.585216,7.892887,10.210053,-4.492510,10.210053,-4.492510,32.175576,100.0
2022-10-19 00:00:00,5.922969,6.254707,5.611613,6.335308,5.611613,6.335308,19.647720,100.0
...,...,...,...,...,...,...,...,...
2010-05-31 00:00:00,4.527580,5.486061,-3.690799,7.968227,-3.690799,7.968227,14.386604,100.0
2010-03-29 00:00:00,-6.189211,11.298808,1.862514,-4.347457,1.862514,-4.347457,4.135807,100.0
2010-01-25 00:00:00,6.337736,-9.060182,9.627639,-0.401594,9.627639,-0.401594,9.092972,100.0
2009-11-24 00:00:00,15.595328,6.499772,-8.924138,5.018214,-8.924138,5.018214,11.036812,100.0


In [ ]:
avg_profit_df4.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\year low sort, RS EMA20rate大於80, RS EMA250rate大於75, Price大於50MA, Volume 50MA大於100k.xlsx')

In [ ]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-5002, -30, 62)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    f = 1
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if f == 1:
                allstock = stockdata
                f = 0
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            # print(e)
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS EMA250rate>85'] = (allstock.loc[id, 'RS EMA250rate']>85)
            allstock.loc[id, 'RS EMA20rate>85'] = (allstock.loc[id, 'RS EMA20rate'] > 85)
            allstock.loc[id, 'RS EMA250rate>70'] = (allstock.loc[id, 'RS EMA250rate']>70)
            allstock.loc[id, 'RS EMA20rate>75'] = (allstock.loc[id, 'RS EMA20rate'] > 75)
            allstock.loc[id, 'ATR250/price'] = (allstock.loc[id, 'ATR250'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR50/price'] = (allstock.loc[id, 'ATR50'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR20/price'] = (allstock.loc[id, 'ATR20'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR250/price<0.03'] = (allstock.loc[id, 'ATR250/price'] < 0.03)
            allstock.loc[id, 'ATR250/price<0.15'] = (allstock.loc[id, 'ATR250/price'] < 0.15)
            allstock.loc[id, 'ATR50/price<0.03'] = (allstock.loc[id, 'ATR50/price'] < 0.03)
            allstock.loc[id, 'ATR20/price<0.03'] = (allstock.loc[id, 'ATR20/price'] < 0.03)            
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 250*1000)
            allstock.loc[id, 'Volume 50MA>1000k'] = (allstock.loc[id, 'Volume 50MA'] > 1000*1000)
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
            allstock.loc[id, 'T1'] = all(allstock.loc[id, ['RS EMA250rate>85','RS EMA20rate>85', 'Volume 50MA>100k']])
            allstock.loc[id, 'T2'] = all(allstock.loc[id, ['ATR250/price<0.15', 'Volume 50MA>1000k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    template1_df = allstock.loc[allstock['T1'], ['產業別', 'T1', 'T2', 'ATR250/price', 'ATR50/price', 'ATR20/price'
                                                 , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                                 , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA'
                                                 ,'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    template2_df = allstock.loc[allstock['T2'], ['產業別', 'T1', 'T2', 'ATR250/price', 'ATR50/price', 'ATR20/price'
                                                 , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                                 , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA'
                                                 ,'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    template1_df.dropna(inplace=True)
    template2_df.dropna(inplace=True)
    template1_df = template1_df.sort_values(by='RS EMA250rate', ascending=True).iloc[-80:]
    template2_num = int(len(template1_df.index.values)/4)
    template2_df = template2_df.sort_values(by='RS EMA250rate', ascending=True)
    print(f'T2 len:{len(template2_df.index.values)}')
    template2_df = template2_df.loc[~template2_df.index.isin(template1_df.index)].iloc[-template2_num:]
    apexstock = pd.concat([template1_df, template2_df])
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(f'T1 len:{len(template1_df.index.values)}')
    print(f'T2 len:{len(template2_df.index.values)}')
    print(f'apexstock len:{len(apexstock.index.values)}')
    print(str(day))
    

    profit_list = []
    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+62))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        ID = str(int(ID))
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
                try:
                    apexstock.drop(ID, inplace=True)
                except:
                    pass
                continue
        try:
            initial_price = stock.loc[stock.index>day, 'Adj Close'].iloc[0]
            final_price = stock.loc[stock.index<day2, 'Adj Close'].iloc[-1]
            profit_list.append([ID, (final_price-initial_price)/initial_price])
        except Exception as e:
            print(e)
            try:
                apexstock.drop(ID, inplace=True)
            except:
                pass
            continue

    print(len(apexstock.index.values))
    print(np.shape(profit_list))
    profit_df = pd.DataFrame(profit_list, columns=['ID', 'profit'])
    profit_df.set_index('ID', inplace=True)
    apexstock.loc[apexstock.index.values, 'profit'] = profit_df['profit'].values
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/ETF test/stock choose T7/' + str(day).split(' ')[0] + 'RS選股(T7)' + '.xlsx', encoding='utf-8-sig')
    profit = {'season1_profit':profit_df.sum().values,
            'number of stock':len(apexstock.index.values)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        day_profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([day_profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

In [4]:
avg_profit_df5 = pd.DataFrame([], columns=all_df.columns)
for ind in all_df.index.values:
    avg_profit_df5.loc[ind] = 100*all_df.loc[ind]/all_df.loc[ind, 'number of stock']
avg_profit_df5.loc['total'] = avg_profit_df5.sum()
display(avg_profit_df5)
avg_profit_df5.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\T7.xlsx')

,season1_profit,number of stock
2023-06-26 00:00:00,-1.631262,100.0
2023-04-24 00:00:00,18.098566,100.0
2023-02-20 00:00:00,8.609349,100.0
2022-12-19 00:00:00,12.466069,100.0
2022-10-18 00:00:00,8.932519,100.0
...,...,...
2010-05-28 00:00:00,5.125907,100.0
2010-03-29 00:00:00,-8.678513,100.0
2010-01-25 00:00:00,11.074321,100.0
2009-11-23 00:00:00,13.625859,100.0


In [6]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-5002, -30, 62)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    f = 1
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if f == 1:
                allstock = stockdata
                f = 0
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            # print(e)
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS EMA250rate>85'] = (allstock.loc[id, 'RS EMA250rate']>85)
            allstock.loc[id, 'RS EMA20rate>85'] = (allstock.loc[id, 'RS EMA20rate'] > 85)
            allstock.loc[id, 'RS EMA250rate>75'] = (allstock.loc[id, 'RS EMA250rate']>75)
            allstock.loc[id, 'RS EMA20rate<99'] = (allstock.loc[id, 'RS EMA20rate'] < 99)
            allstock.loc[id, 'RS EMA250rate>70'] = (allstock.loc[id, 'RS EMA250rate']>70)
            allstock.loc[id, 'RS EMA20rate>75'] = (allstock.loc[id, 'RS EMA20rate'] > 75)
            allstock.loc[id, 'ATR250/price'] = (allstock.loc[id, 'ATR250'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR50/price'] = (allstock.loc[id, 'ATR50'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR20/price'] = (allstock.loc[id, 'ATR20'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR250/price<0.03'] = (allstock.loc[id, 'ATR250/price'] < 0.03)
            allstock.loc[id, 'ATR250/price<0.5'] = (allstock.loc[id, 'ATR250/price'] < 0.15)
            allstock.loc[id, 'ATR50/price<0.03'] = (allstock.loc[id, 'ATR50/price'] < 0.03)
            allstock.loc[id, 'ATR20/price<0.03'] = (allstock.loc[id, 'ATR20/price'] < 0.03)            
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 250*1000)
            allstock.loc[id, 'Volume 50MA>1000k'] = (allstock.loc[id, 'Volume 50MA'] > 1000*1000)
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
            allstock.loc[id, 'T1'] = any([all(allstock.loc[id, ['RS EMA250rate>85','RS EMA20rate>85', 'RS EMA20rate<99', 'Volume 50MA>100k']]), all(allstock.loc[id, ['RS EMA250rate>75','RS 250EMA is 10MAX', 'RS EMA20rate<99', 'Volume 50MA>100k']])])
            allstock.loc[id, 'T2'] = all(allstock.loc[id, ['ATR250/price<0.5', 'Volume 50MA>1000k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    template1_df = allstock.loc[allstock['T1'], ['產業別', 'T1', 'T2', 'ATR250/price', 'ATR50/price', 'ATR20/price','RS 250EMA is 10MAX'
                                                 , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                                 , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA'
                                                 ,'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    template2_df = allstock.loc[allstock['T2'], ['產業別', 'T1', 'T2', 'ATR250/price', 'ATR50/price', 'ATR20/price'
                                                 , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                                 , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA'
                                                 ,'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    template1_df.dropna(inplace=True)
    apexstock = template1_df.sort_values(by='RS EMA250rate', ascending=True)
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(f'apexstock len:{len(apexstock.index.values)}')
    print(str(day))
    
    # template1_df.dropna(inplace=True)
    # template2_df.dropna(inplace=True)
    # template1_df = template1_df.sort_values(by='RS MA250rate', ascending=True)
    # template2_num = int(len(template1_df.index.values)/4)
    # template2_df = template2_df.sort_values(by='RS MA250rate', ascending=True)
    # print(f'T2 len:{len(template2_df.index.values)}')
    # template2_df = template2_df.loc[~template2_df.index.isin(template1_df.index)].iloc[-template2_num:]
    # apexstock = pd.concat([template1_df, template2_df])
    # apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    # apexstock = apexstock.dropna()
    # print(f'T1 len:{len(template1_df.index.values)}')
    # print(f'T2 len:{len(template2_df.index.values)}')
    print(f'apexstock len:{len(apexstock.index.values)}')
    print(str(day))
    

    profit_list = []
    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+62))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        ID = str(int(ID))
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
                try:
                    apexstock.drop(ID, inplace=True)
                except:
                    pass
                continue
        try:
            initial_price = stock.loc[stock.index>day, 'Adj Close'].iloc[0]
            final_price = stock.loc[stock.index<day2, 'Adj Close'].iloc[-1]
            profit_list.append([ID, (final_price-initial_price)/initial_price])
        except Exception as e:
            print(e)
            try:
                apexstock.drop(ID, inplace=True)
            except:
                pass
            continue

    print(len(apexstock.index.values))
    print(np.shape(profit_list))
    profit_df = pd.DataFrame(profit_list, columns=['ID', 'profit'])
    profit_df.set_index('ID', inplace=True)
    apexstock.loc[apexstock.index.values, 'profit'] = profit_df['profit'].values
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/ETF test/stock choose T10/' + str(day).split(' ')[0] + 'RS選股(T10)' + '.xlsx', encoding='utf-8-sig')
    profit = {'season1_profit':profit_df.sum().values,
            'number of stock':len(apexstock.index.values)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        day_profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([day_profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

     有價證券名稱 市場別    產業別 公開發行/上市(櫃)/發行日
ID                                   
1101     台泥  上市   水泥工業     1962/02/09
1102     亞泥  上市   水泥工業     1962/06/08
1103     嘉泥  上市   水泥工業     1969/11/14
1104     環泥  上市   水泥工業     1971/02/01
1108     幸福  上市   水泥工業     1990/06/06
...     ...  ..    ...            ...
9949     琉園  上櫃  文化創意業     2003/11/21
9950    萬國通  上櫃   塑膠工業     2004/02/17
9951     皇田  上櫃   電機機械     2003/12/18
9960    邁達康  上櫃   運動休閒     2004/12/06
9962     有益  上櫃   鋼鐵工業     2006/07/10

[1788 rows x 4 columns]
day 2009-11-24 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:74
2009-11-24 00:00:00
apexstock len:74
2009-11-24 00:00:00
74
(74, 2)


,season1_profit,number of stock
2009-11-24,12.61007,74


day 2010-01-25 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:68
2010-01-25 00:00:00
apexstock len:68
2010-01-25 00:00:00
68
(68, 2)


,season1_profit,number of stock
2010-01-25,6.822703,68
2009-11-24,12.610070,74


day 2010-03-29 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:70
2010-03-29 00:00:00
apexstock len:70
2010-03-29 00:00:00
70
(70, 2)


...

,season1_profit,number of stock
2023-02-20,9.036548,111
2022-12-20,9.259216,86
2022-10-19,6.680693,78
2022-08-18,-14.942113,107
2022-06-17,12.643850,104
...,...,...
2010-07-30,1.388660,66
2010-05-31,4.807526,77
2010-03-29,-5.191282,70
2010-01-25,6.822703,68


day 2023-04-24 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:138
2023-04-24 00:00:00
apexstock len:138
2023-04-24 00:00:00
138
(138, 2)


,season1_profit,number of stock
2023-04-24,17.683279,138
2023-02-20,9.036548,111
2022-12-20,9.259216,86
2022-10-19,6.680693,78
2022-08-18,-14.942113,107
...,...,...
2010-07-30,1.388660,66
2010-05-31,4.807526,77
2010-03-29,-5.191282,70
2010-01-25,6.822703,68


day 2023-06-26 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:134
2023-06-26 00:00:00
apexstock len:134
2023-06-26 00:00:00
134
(134, 2)


,season1_profit,number of stock
2023-06-26,1.123873,134
2023-04-24,17.683279,138
2023-02-20,9.036548,111
2022-12-20,9.259216,86
2022-10-19,6.680693,78
...,...,...
2010-07-30,1.388660,66
2010-05-31,4.807526,77
2010-03-29,-5.191282,70
2010-01-25,6.822703,68


In [7]:
display(all_df)
avg_profit_df6 = pd.DataFrame([])
for ind in all_df.index.values:
    avg_profit_df6.loc[ind, 'season1 profit'] = 100*all_df.loc[ind, 'season1_profit']/all_df.loc[ind, 'number of stock']
    avg_profit_df6.loc[ind, 'numbers of stock'] = all_df.loc[ind, 'number of stock']
avg_profit_df6.loc['total'] = avg_profit_df6.sum()
display(avg_profit_df6)
avg_profit_df6.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\T10.xlsx')

,season1_profit,number of stock
2023-06-26,1.123873,134
2023-04-24,17.683279,138
2023-02-20,9.036548,111
2022-12-20,9.259216,86
2022-10-19,6.680693,78
...,...,...
2010-07-30,1.388660,66
2010-05-31,4.807526,77
2010-03-29,-5.191282,70
2010-01-25,6.822703,68


,season1 profit,numbers of stock
2023-06-26 00:00:00,0.838711,134.0
2023-04-24 00:00:00,12.813971,138.0
2023-02-20 00:00:00,8.141034,111.0
2022-12-20 00:00:00,10.766530,86.0
2022-10-19 00:00:00,8.564991,78.0
...,...,...
2010-05-31 00:00:00,6.243540,77.0
2010-03-29 00:00:00,-7.416118,70.0
2010-01-25 00:00:00,10.033387,68.0
2009-11-24 00:00:00,17.040635,74.0


In [8]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-5002, -30, 62)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    f = 1
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if f == 1:
                allstock = stockdata
                f = 0
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            # print(e)
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS MA250rate<75'] = (allstock.loc[id, 'RS 250rate']<75)
            allstock.loc[id, 'RS MA250rate>55'] = (allstock.loc[id, 'RS 250rate']>55)
            allstock.loc[id, 'RS MA20rate>85'] = (allstock.loc[id, 'RS 20rate'] > 85)
            allstock.loc[id, 'RS EMA250rate>75'] = (allstock.loc[id, 'RS EMA250rate']>75)
            allstock.loc[id, 'RS EMA20rate<99'] = (allstock.loc[id, 'RS EMA20rate'] < 99)
            allstock.loc[id, 'RS EMA250rate>70'] = (allstock.loc[id, 'RS EMA250rate']>70)
            allstock.loc[id, 'RS EMA20rate>75'] = (allstock.loc[id, 'RS EMA20rate'] > 75)
            allstock.loc[id, 'ATR250/price'] = (allstock.loc[id, 'ATR250'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR50/price'] = (allstock.loc[id, 'ATR50'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR20/price'] = (allstock.loc[id, 'ATR20'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR250/price<0.03'] = (allstock.loc[id, 'ATR250/price'] < 0.03)
            allstock.loc[id, 'ATR250/price<0.5'] = (allstock.loc[id, 'ATR250/price'] < 0.15)
            allstock.loc[id, 'ATR50/price<0.03'] = (allstock.loc[id, 'ATR50/price'] < 0.03)
            allstock.loc[id, 'ATR20/price<0.03'] = (allstock.loc[id, 'ATR20/price'] < 0.03)            
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 250*1000)
            allstock.loc[id, 'Volume 50MA>1000k'] = (allstock.loc[id, 'Volume 50MA'] > 1000*1000)
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
            allstock.loc[id, 'T1'] = all(allstock.loc[id, ['year low sort', 'year high sort', 'RS MA20rate>85', 'RS MA250rate<75', 'RS MA250rate>55', 'Volume 50MA>100k']])
            allstock.loc[id, 'T2'] = all(allstock.loc[id, ['ATR250/price<0.5', 'Volume 50MA>1000k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    template1_df = allstock.loc[allstock['T1'], ['產業別', 'T1', 'T2', 'ATR250/price', 'ATR50/price', 'ATR20/price','RS 250EMA is 10MAX'
                                                 , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                                 , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA'
                                                 ,'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    template2_df = allstock.loc[allstock['T2'], ['產業別', 'T1', 'T2', 'ATR250/price', 'ATR50/price', 'ATR20/price'
                                                 , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                                 , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA'
                                                 ,'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    template1_df.dropna(inplace=True)
    apexstock = template1_df.sort_values(by='RS EMA250rate', ascending=True)
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(f'apexstock len:{len(apexstock.index.values)}')
    print(str(day))
    
    # template1_df.dropna(inplace=True)
    # template2_df.dropna(inplace=True)
    # template1_df = template1_df.sort_values(by='RS MA250rate', ascending=True)
    # template2_num = int(len(template1_df.index.values)/4)
    # template2_df = template2_df.sort_values(by='RS MA250rate', ascending=True)
    # print(f'T2 len:{len(template2_df.index.values)}')
    # template2_df = template2_df.loc[~template2_df.index.isin(template1_df.index)].iloc[-template2_num:]
    # apexstock = pd.concat([template1_df, template2_df])
    # apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    # apexstock = apexstock.dropna()
    # print(f'T1 len:{len(template1_df.index.values)}')
    # print(f'T2 len:{len(template2_df.index.values)}')
    print(f'apexstock len:{len(apexstock.index.values)}')
    print(str(day))
    

    profit_list = []
    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+62))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        ID = str(int(ID))
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
                try:
                    apexstock.drop(ID, inplace=True)
                except:
                    pass
                continue
        try:
            initial_price = stock.loc[stock.index>day, 'Adj Close'].iloc[0]
            final_price = stock.loc[stock.index<day2, 'Adj Close'].iloc[-1]
            profit_list.append([ID, (final_price-initial_price)/initial_price])
        except Exception as e:
            print(e)
            try:
                apexstock.drop(ID, inplace=True)
            except:
                pass
            continue

    print(len(apexstock.index.values))
    print(np.shape(profit_list))
    profit_df = pd.DataFrame(profit_list, columns=['ID', 'profit'])
    profit_df.set_index('ID', inplace=True)
    apexstock.loc[apexstock.index.values, 'profit'] = profit_df['profit'].values
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/ETF test/stock choose T5/' + str(day).split(' ')[0] + 'RS選股(T5)' + '.xlsx', encoding='utf-8-sig')
    profit = {'season1_profit':profit_df.sum().values,
            'number of stock':len(apexstock.index.values)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        day_profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([day_profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

     有價證券名稱 市場別    產業別 公開發行/上市(櫃)/發行日
ID                                   
1101     台泥  上市   水泥工業     1962/02/09
1102     亞泥  上市   水泥工業     1962/06/08
1103     嘉泥  上市   水泥工業     1969/11/14
1104     環泥  上市   水泥工業     1971/02/01
1108     幸福  上市   水泥工業     1990/06/06
...     ...  ..    ...            ...
9949     琉園  上櫃  文化創意業     2003/11/21
9950    萬國通  上櫃   塑膠工業     2004/02/17
9951     皇田  上櫃   電機機械     2003/12/18
9960    邁達康  上櫃   運動休閒     2004/12/06
9962     有益  上櫃   鋼鐵工業     2006/07/10

[1788 rows x 4 columns]
day 2009-11-24 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:3
2009-11-24 00:00:00
apexstock len:3
2009-11-24 00:00:00
3
(3, 2)


,season1_profit,number of stock
2009-11-24,0.723287,3


day 2010-01-25 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:5
2010-01-25 00:00:00
apexstock len:5
2010-01-25 00:00:00
5
(5, 2)


,season1_profit,number of stock
2010-01-25,0.361889,5
2009-11-24,0.723287,3


day 2010-03-29 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:10
2010-03-29 00:00:00
apexstock len:10
2010-03-29 00:00:00
10
(10, 2)


...

,season1_profit,number of stock
2023-02-20,0.932511,19
2022-12-20,1.108536,17
2022-10-19,0.232661,4
2022-08-18,-1.095739,12
2022-06-17,1.301604,14
...,...,...
2010-07-30,0.059483,2
2010-05-31,1.007894,5
2010-03-29,-0.835927,10
2010-01-25,0.361889,5


day 2023-04-24 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:26
2023-04-24 00:00:00
apexstock len:26
2023-04-24 00:00:00
26
(26, 2)


,season1_profit,number of stock
2023-04-24,6.334378,26
2023-02-20,0.932511,19
2022-12-20,1.108536,17
2022-10-19,0.232661,4
2022-08-18,-1.095739,12
...,...,...
2010-07-30,0.059483,2
2010-05-31,1.007894,5
2010-03-29,-0.835927,10
2010-01-25,0.361889,5


day 2023-06-26 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:30
2023-06-26 00:00:00
apexstock len:30
2023-06-26 00:00:00
30
(30, 2)


,season1_profit,number of stock
2023-06-26,3.270473,30
2023-04-24,6.334378,26
2023-02-20,0.932511,19
2022-12-20,1.108536,17
2022-10-19,0.232661,4
...,...,...
2010-07-30,0.059483,2
2010-05-31,1.007894,5
2010-03-29,-0.835927,10
2010-01-25,0.361889,5


In [9]:
display(all_df)
avg_profit_df7 = pd.DataFrame([])
for ind in all_df.index.values:
    avg_profit_df7.loc[ind, 'season1 profit'] = 100*all_df.loc[ind, 'season1_profit']/all_df.loc[ind, 'number of stock']
    avg_profit_df7.loc[ind, 'numbers of stock'] = all_df.loc[ind, 'number of stock']
avg_profit_df7.loc['total'] = avg_profit_df7.sum()
display(avg_profit_df7)
avg_profit_df7.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\T5.xlsx')

,season1_profit,number of stock
2023-06-26,3.270473,30
2023-04-24,6.334378,26
2023-02-20,0.932511,19
2022-12-20,1.108536,17
2022-10-19,0.232661,4
...,...,...
2010-07-30,0.059483,2
2010-05-31,1.007894,5
2010-03-29,-0.835927,10
2010-01-25,0.361889,5


,season1 profit,numbers of stock
2023-06-26 00:00:00,10.901578,30.0
2023-04-24 00:00:00,24.362994,26.0
2023-02-20 00:00:00,4.907953,19.0
2022-12-20 00:00:00,6.520798,17.0
2022-10-19 00:00:00,5.816537,4.0
...,...,...
2010-05-31 00:00:00,20.157887,5.0
2010-03-29 00:00:00,-8.359269,10.0
2010-01-25 00:00:00,7.237773,5.0
2009-11-24 00:00:00,24.109569,3.0
